## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-16-01-17-03 +0000,nyt,"On Fox News, Mamdani Speaks Directly to Trump:...",https://www.nytimes.com/2025/10/15/nyregion/ma...
1,2025-10-16-01-13-47 +0000,nyt,Gavin Newsom Vetoes Reparations Bills in Calif...,https://www.nytimes.com/2025/10/15/us/californ...
2,2025-10-16-01-10-53 +0000,nyt,The Lonely House Republican Still Coming to Wo...,https://www.nytimes.com/2025/10/15/us/politics...
3,2025-10-16-01-10-27 +0000,nypost,Holy mackerel! Falling fish smashes Canadian w...,https://nypost.com/2025/10/15/world-news/falli...
4,2025-10-16-01-03-17 +0000,bbc,Government publishes key witness statements in...,https://www.bbc.com/news/articles/c0ex172rxwzo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2484/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,trump,53
67,new,18
116,hamas,18
50,he,17
21,shutdown,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
128,2025-10-15-19-58-31 +0000,latimes,California judge halts Trump federal job cuts ...,https://www.latimes.com/politics/story/2025-10...,116
30,2025-10-15-23-45-14 +0000,nyt,Judge Temporarily Blocks Trump From Firing Gov...,https://www.nytimes.com/2025/10/15/us/politics...,111
65,2025-10-15-22-10-23 +0000,nypost,Dems hand out vile ‘Is he dead yet?’ bracelets...,https://nypost.com/2025/10/15/us-news/winnebag...,104
113,2025-10-15-20-27-34 +0000,cbc,Judge blocks Trump from firing workers during ...,https://www.cbc.ca/news/world/trump-shutdown-9...,101
115,2025-10-15-20-21-00 +0000,wsj,A federal judge temporarily blocked the Trump ...,https://www.wsj.com/politics/policy/judge-bloc...,101


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
128,116,2025-10-15-19-58-31 +0000,latimes,California judge halts Trump federal job cuts ...,https://www.latimes.com/politics/story/2025-10...
239,69,2025-10-15-14-38-01 +0000,nypost,"Family of kidnapped IDF soldier, 18, only foun...",https://nypost.com/2025/10/15/world-news/idf-s...
215,57,2025-10-15-16-07-50 +0000,nypost,US student group calls for ‘death’ to Zionist ...,https://nypost.com/2025/10/15/world-news/stude...
31,51,2025-10-15-23-41-38 +0000,latimes,Man accused of maliciously igniting Palisades ...,https://www.latimes.com/california/story/2025-...
104,51,2025-10-15-20-52-20 +0000,nyt,Supreme Court Appears Skeptical of Key Provisi...,https://www.nytimes.com/2025/10/15/us/politics...
317,46,2025-10-15-10-00-00 +0000,nypost,Dr. Keith Ablow tells ‘Pod Force One’ how he b...,https://nypost.com/2025/10/15/us-news/dr-keith...
58,38,2025-10-15-22-29-00 +0000,wsj,More than three dozen organizations and indivi...,https://www.wsj.com/politics/policy/trump-ball...
159,37,2025-10-15-18-37-00 +0000,wsj,Lower U.S. Tariffs Would Boost India Growth Ou...,https://www.wsj.com/articles/lower-u-s-tariffs...
27,35,2025-10-15-23-55-36 +0000,nypost,Democratic Rep. Seth Moulton launches primary ...,https://nypost.com/2025/10/15/us-news/democrat...
235,34,2025-10-15-14-52-00 +0000,wsj,BlackRock’s AI Consortium to Buy Aligned Data ...,https://www.wsj.com/finance/investing/blackroc...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
